In [31]:
import os

In [32]:
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = ""
os.environ["OPENAI_API_KEY"] = ""
os.environ["TAVILY_API_KEY"] = ""

In [34]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///path_to_db/my_database.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM base LIMIT 10;")

sqlite
['applprev', 'base', 'credit_bureau_a', 'credit_bureau_b', 'credit_bureau_b_2', 'debitcard', 'deposit', 'other', 'person', 'static', 'static_cb', 'tax_registry_a', 'tax_registry_b', 'tax_registry_c']


"[(0, '2019-01-03 00:00:00.000000', 201901, 0, 0), (1, '2019-01-03 00:00:00.000000', 201901, 0, 0), (2, '2019-01-04 00:00:00.000000', 201901, 0, 0), (3, '2019-01-03 00:00:00.000000', 201901, 0, 0), (4, '2019-01-04 00:00:00.000000', 201901, 0, 1), (5, '2019-01-02 00:00:00.000000', 201901, 0, 0), (6, '2019-01-03 00:00:00.000000', 201901, 0, 0), (7, '2019-01-03 00:00:00.000000', 201901, 0, 0), (8, '2019-01-03 00:00:00.000000', 201901, 0, 0), (9, '2019-01-03 00:00:00.000000', 201901, 0, 0)]"

In [44]:
context = db.get_context()

In [46]:
print(context["table_info"])


CREATE TABLE applprev (
	case_id BIGINT, 
	"max_actualdpd_943P" FLOAT, 
	"max_annuity_853A" FLOAT, 
	"max_credacc_actualbalance_314A" FLOAT, 
	"max_credacc_credlmt_575A" FLOAT, 
	"max_credacc_maxhisbal_375A" FLOAT, 
	"max_credacc_minhisbal_90A" FLOAT, 
	"max_credamount_590A" FLOAT, 
	"max_currdebt_94A" FLOAT, 
	"max_downpmt_134A" FLOAT, 
	"max_mainoccupationinc_437A" FLOAT, 
	"max_maxdpdtolerance_577P" FLOAT, 
	"max_outstandingdebt_522A" FLOAT, 
	"max_revolvingaccount_394A" FLOAT, 
	"max_approvaldate_319D" DATETIME, 
	"max_creationdate_885D" DATETIME, 
	"max_dateactivated_425D" DATETIME, 
	"max_dtlastpmt_581D" DATETIME, 
	"max_dtlastpmtallstes_3545839D" DATETIME, 
	"max_employedfrom_700D" DATETIME, 
	"max_firstnonzeroinstldate_307D" DATETIME, 
	"max_cancelreason_3545846M" TEXT, 
	"max_district_544M" TEXT, 
	"max_education_1138M" TEXT, 
	"max_postype_4733339M" TEXT, 
	"max_profession_152M" TEXT, 
	"max_rejectreason_755M" TEXT, 
	"max_rejectreasonclient_4145042M" TEXT, 
	"max_byoccupati

## **CREATE SQL AGENT**

In [35]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

## RAG

In [51]:
agent_executor.invoke(
    "from all the table name base, if I were to create a ml model which columns should be relevant."
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


applprev, base, credit_bureau_a, credit_bureau_b, credit_bureau_b_2, debitcard, deposit, other, person, static, static_cb, tax_registry_a, tax_registry_b, tax_registry_c
Invoking: `sql_db_schema` with `{'table_names': 'base, credit_bureau_a, debitcard, deposit, person, static, static_cb, tax_registry_a, tax_registry_b, tax_registry_c'}`



CREATE TABLE base (
	case_id BIGINT, 
	date_decision DATETIME, 
	"MONTH" BIGINT, 
	"WEEK_NUM" BIGINT, 
	target BIGINT
)

/*
3 rows from base table:
case_id	date_decision	MONTH	WEEK_NUM	target
0	2019-01-03 00:00:00	201901	0	0
1	2019-01-03 00:00:00	201901	0	0
2	2019-01-04 00:00:00	201901	0	0
*/


CREATE TABLE credit_bureau_a (
	case_id BIGINT, 
	"max_credlmt_230A" FLOAT, 
	"max_credlmt_935A" FLOAT, 
	"max_debtoutstand_525A" FLOAT, 
	"max_debtoverdue_47A" FLOAT, 
	"max_dpdmax_139P" FLOAT, 
	"max_dpdmax_757P" FLOAT, 
	"max_instlamount_768A" FLOAT, 
	"max_instlamount_

{'input': 'from all the table name base, if I were to create a ml model which columns should be relevant.',
 'output': 'Based on the tables in the database, relevant columns for creating a machine learning model could be:\n\n1. From the "base" table:\n   - case_id\n   - date_decision\n   - "MONTH"\n   - "WEEK_NUM"\n   - target\n\n2. From the "credit_bureau_a" table:\n   - case_id\n   - Various columns related to credit limits, debt outstanding, overdue amounts, dates, rates, and counts\n\n3. From the "debitcard" table:\n   - case_id\n   - "max_last180dayaveragebalance_704A"\n   - "max_last180dayturnover_1134A"\n   - "max_last30dayturnover_651A"\n   - "max_openingdate_857D"\n\n4. From the "deposit" table:\n   - case_id\n   - "max_amount_416A"\n   - "max_contractenddate_991D"\n   - "max_openingdate_313D"\n\n5. From the "person" table:\n   - case_id\n   - Various columns related to occupation, birthdate, addresses, education, marital status, gender, income type, and more\n\n6. From the "s

## Table QA

In [54]:
agent_executor.invoke(
    "how many row in column target that has value = 0"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


applprev, base, credit_bureau_a, credit_bureau_b, credit_bureau_b_2, debitcard, deposit, other, person, static, static_cb, tax_registry_a, tax_registry_b, tax_registry_c
Invoking: `sql_db_schema` with `{'table_names': 'base'}`



CREATE TABLE base (
	case_id BIGINT, 
	date_decision DATETIME, 
	"MONTH" BIGINT, 
	"WEEK_NUM" BIGINT, 
	target BIGINT
)

/*
3 rows from base table:
case_id	date_decision	MONTH	WEEK_NUM	target
0	2019-01-03 00:00:00	201901	0	0
1	2019-01-03 00:00:00	201901	0	0
2	2019-01-04 00:00:00	201901	0	0
*/
Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(*) FROM base WHERE target = 0'}`


[(1478665,)]There are 1,478,665 rows in the column "target" that have the value of 0.

> Finished chain.


{'input': 'how many row in column target that has value = 0',
 'output': 'There are 1,478,665 rows in the column "target" that have the value of 0.'}